In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=34af4d3a5273a50dc4a8975326931c793cf8db9085f08e905ba877963649962a
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType

# Start Spark session
spark = SparkSession.builder.appName("FraudDetectionStreaming").getOrCreate()

# Define schema for the transaction data
schema = StructType([
    StructField("TransactionID", IntegerType(), True),
    StructField("UserID", IntegerType(), True),
    StructField("TransactionAmount", DoubleType(), True),
    StructField("TransactionLocation", StringType(), True),
    StructField("TransactionTime", TimestampType(), True),
    StructField("PaymentMethod", StringType(), True),
    StructField("TransactionStatus", StringType(), True),
    StructField("FraudLabel", IntegerType(), True)
])

# Reading streaming source
streaming_df = spark.readStream.schema(schema).csv("/content/sample_data/transaction.csv")

# Process the stream 
query = streaming_df.writeStream.outputMode("append").format("console").start()

print("Streaming stared...")
#query.awaitTermination()


Streaming stared...


In [5]:
from pyspark.sql.functions import col

# Define threshold for flagging large transactions
large_transaction_threshold = 1000.0

# Filter transactions that exceed the threshold and are flagged as fraud
anomalous_transactions = streaming_df.filter(
    (col("TransactionAmount") > large_transaction_threshold) |
    (col("FraudLabel") == 1)
)

# Output anomalous transactions to console 
anomaly_query = anomalous_transactions.writeStream.outputMode("append").format("console").start()

print("Anomaly detection started...")
#anomaly_query.awaitTermination()


Anomaly detection started...
